In [3]:
!pip install flask-ngrok

In [7]:
!python /content/cardio/main.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://cd895b2de587.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
^C


In [ ]:
#app.py
from flask import Flask

UPLOAD_FOLDER = 'static/uploads/'

app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

In [ ]:
#main.py
import os
from app import app
import urllib.request
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok

run_with_ngrok(app)
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
	
@app.route('/')
def upload_form():
	return render_template('upload.html')

@app.route('/', methods=['POST'])
def upload_image():
	if 'file' not in request.files:
		flash('No file part')
		return redirect(request.url)
	file = request.files['file']
	if file.filename == '':
		flash('No image selected for uploading')
		return redirect(request.url)
	if file and allowed_file(file.filename):
		filename = secure_filename(file.filename)
		file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
		#print('upload_image filename: ' + filename)
		flash('Image successfully uploaded and displayed below')
		return render_template('upload.html', filename=filename)
	else:
		flash('Allowed image types are -> png, jpg, jpeg, gif')
		return redirect(request.url)

@app.route('/display/<filename>')
def display_image(filename):
	#print('display_image filename: ' + filename)
	return redirect(url_for('static', filename='uploads/' + filename), code=301)

if __name__ == "__main__":
    app.run()

In [ ]:
#in templates of cardio of contents
#upload.html
<!doctype html>
<title>Python Flask File Upload Example</title>
<h2>Select a file to upload</h2>
<p>
	{% with messages = get_flashed_messages() %}
	  {% if messages %}
		<ul>
		{% for message in messages %}
		  <li>{{ message }}</li>
		{% endfor %}
		</ul>
	  {% endif %}
	{% endwith %}
</p>
{% if filename %}
	<div>
		<img src="{{ url_for('display_image', filename=filename) }}">
	</div>
{% endif %}
<form method="post" action="/" enctype="multipart/form-data">
    <dl>
		<p>
			<input type="file" name="file" autocomplete="off" required>
		</p>
    </dl>
    <p>
		<input type="submit" value="Submit">
	</p>
</form>

In [8]:
!zip -r /content/file.zip /content/cardio

  adding: content/cardio/ (stored 0%)
  adding: content/cardio/app.py (deflated 28%)
  adding: content/cardio/main.py (deflated 56%)
  adding: content/cardio/__pycache__/ (stored 0%)
  adding: content/cardio/__pycache__/app.cpython-37.pyc (deflated 15%)
  adding: content/cardio/.ipynb_checkpoints/ (stored 0%)
  adding: content/cardio/templates/ (stored 0%)
  adding: content/cardio/templates/upload.html (deflated 44%)


In [9]:
from google.colab import files

In [10]:
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>